In [1]:
import numpy as np
import pandas as pd

In [2]:
part1=pd.read_csv('RAWDATA/training_set_3ap_loc30_nav82.csv')
# part2=pd.read_csv('RAWDATA/training_set_3ap_loc30_nav86.csv')  #ap_from_ap_0_rssi全部为空，所以不用于模拟三ap数据
part3=pd.read_csv('RAWDATA/training_set_3ap_loc31_nav82.csv')
part4=pd.read_csv('RAWDATA/training_set_3ap_loc31_nav86.csv')
part5=pd.read_csv('RAWDATA/training_set_3ap_loc32_nav82.csv')
part6=pd.read_csv('RAWDATA/training_set_3ap_loc32_nav86.csv')
part7=pd.read_csv('RAWDATA/training_set_3ap_loc33_nav82_new.csv')
part8=pd.read_csv('RAWDATA/training_set_3ap_loc33_nav88.csv')

resource=pd.concat([part1,part3,part4,part5,part6,part7,part8],axis=0)

In [3]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
Index: 707 entries, 0 to 98
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   test_id                      707 non-null    int64  
 1   test_dur                     707 non-null    int64  
 2   loc_id                       707 non-null    object 
 3   protocol                     707 non-null    object 
 4   pkt_len                      707 non-null    int64  
 5   bss_id                       707 non-null    int64  
 6   ap_name                      707 non-null    object 
 7   ap_mac                       707 non-null    object 
 8   ap_id                        707 non-null    object 
 9   pd                           707 non-null    int64  
 10  ed                           707 non-null    int64  
 11  nav                          707 non-null    int64  
 12  eirp                         707 non-null    int64  
 13  ap_from_ap_0_sum_ant_rssi 

In [4]:
columns=[0,1,2,4,6,7,8,12,22,23,25,26,28,29,31,32,34,35,37,38,40,41,50,53,54]  #这里没删bss_id，为了后续合并空白值计算

resource=resource.drop(resource.columns[columns],axis=1)
resource.index=range(resource.shape[0])

In [5]:
#resource.to_excel('temp.xlsx')

In [6]:
resource=resource.drop(range(551,608),axis=0)  #删除指定有空值的行
resource.index=range(resource.shape[0])

In [7]:
for i in range(resource.shape[0]):
    if resource.iloc[i,list(resource.columns).index('bss_id')]==2:
        continue
    else: 
        if resource.iloc[i,list(resource.columns).index('bss_id')]==0:
            resource.iloc[i,list(resource.columns).index('ap_from_ap_0_sum_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_1_sum_ant_rssi')]
            resource.iloc[i,list(resource.columns).index('ap_from_ap_0_max_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_1_max_ant_rssi')]
            resource.iloc[i,list(resource.columns).index('ap_from_ap_0_mean_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_1_mean_ant_rssi')]
            resource.iloc[i,list(resource.columns).index('sta_from_sta_0_rssi')]=resource.iloc[i,list(resource.columns).index('sta_from_sta_1_rssi')]
        resource.iloc[i,list(resource.columns).index('ap_from_ap_1_sum_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_2_sum_ant_rssi')]
        resource.iloc[i,list(resource.columns).index('ap_from_ap_1_max_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_2_max_ant_rssi')]
        resource.iloc[i,list(resource.columns).index('ap_from_ap_1_mean_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_2_mean_ant_rssi')]
        resource.iloc[i,list(resource.columns).index('sta_from_sta_1_rssi')]=resource.iloc[i,list(resource.columns).index('sta_from_sta_2_rssi')]

In [8]:
resource=resource.drop(['ap_from_ap_2_sum_ant_rssi',
                       'ap_from_ap_2_max_ant_rssi','ap_from_ap_2_mean_ant_rssi','sta_from_sta_2_rssi'],axis=1)

In [9]:
# 定义管理rssi中异常值数据，并创造出新特征值
def cal_pd(apmax,pd):
    apmax = eval(apmax)
    comparison_result = [1 if x >= pd else 0 for x in apmax]
    proportion_of_ones = sum(comparison_result) / len(apmax)
    return proportion_of_ones

def cal_ed(apmax,ed):
    apmax = eval(apmax)
    comparison_result = [1 if x >= ed else 0 for x in apmax]
    proportion_of_ones = sum(comparison_result) / len(apmax)
    return proportion_of_ones

def cal_nav(apmean,nav):
    apmean = eval(apmean)
    comparison_result = [1 if x >= nav else 0 for x in apmean]
    proportion_of_ones = sum(comparison_result) / len(apmean)
    return proportion_of_ones    


def cal_amsdu(seq_time, ppdu_dur, ampdu, per, throughput):
    # 计量单位：微秒 us
    # 一次发包成功总耗时为RTS+SIFS+CTS+SIFS+ppdu_dur
    # ppdu为一个数据帧的时长，即从RTS-CTS后发出数据帧到收到该帧的ACK
    success_dur = ppdu_dur
    # 一次发包失败总耗时为RTS+ACK_timeout = 85us
    fail_dur = 85.0

    # 假设成功发出x个ppdu包， 则有 per/(1-per) * x 个失败包
    # 总共耗时seq_time = per/(1-per) * x *fail_dur + x * success_dur
    x = seq_time / (success_dur + (per / (1-per) * fail_dur))
    """
    在802.11 WLAN中，最常见的PHY头部大小通常是160位（20字节），适用于802.11a、802.11g、802.11n、802.11ac和802.11ax等标准。这是因为这些标准大多数都基于OFDM（正交频分复用）调制，采用相同的PHY头部结构。
    对于802.11b（DSSS和CCK），虽然它的PHY头部是192位（24字节），但在现代无线网络中，802.11a/g/n/ac/ax的使用更为广泛，因此160位的PHY头部是最常见的。
    """
    pkg_len = 1500
    # 有效吞吐量（Mbps）为 x*effective_ppdu_len*8 / (60 * 1e6)
    # throughput = (x * effective_ppdu_len * 8.0) / (60 * 1e6)
    ppdu_len = throughput * 60 * 1e6 / (x*8)
    amsdu = ((ppdu_len-20) / ampdu - 40) / 1524
    return amsdu

In [10]:
resource['pro_pd_0'] = resource.apply(lambda row: cal_pd(row['ap_from_ap_0_max_ant_rssi'], row['pd']), axis=1)
resource['pro_ed_0'] = resource.apply(lambda row: cal_ed(row['ap_from_ap_0_max_ant_rssi'], row['ed']), axis=1)
resource['pro_nav_0'] = resource.apply(lambda row: cal_nav(row['ap_from_ap_0_mean_ant_rssi'], row['nav']), axis=1)

resource['pro_pd_1'] = resource.apply(lambda row: cal_pd(row['ap_from_ap_1_max_ant_rssi'], row['pd']), axis=1)
resource['pro_ed_1'] = resource.apply(lambda row: cal_ed(row['ap_from_ap_1_max_ant_rssi'], row['ed']), axis=1)
resource['pro_nav_1'] = resource.apply(lambda row: cal_nav(row['ap_from_ap_1_mean_ant_rssi'], row['nav']), axis=1)

In [11]:
drop_columns=['pd','ed','ap_from_ap_0_mean_ant_rssi','ap_from_ap_0_max_ant_rssi',
              'ap_from_ap_1_mean_ant_rssi','ap_from_ap_1_max_ant_rssi']

resource=resource.drop(drop_columns,axis=1)

In [12]:
# 对RSSI求平均值处理
for i in range(3,11):
    resource.iloc[:,i]=resource.iloc[:,i].apply(lambda x: sum(eval(x))/len(eval(x)))

In [13]:
nss_mcs_dict=dict()
phy=[8.6,17.2,25.8,34.4,51.6,68.8,77.4,86.0,103.2,114.7,129.0,143.4,17.2,34.4,51.6,68.8,103.2,137.6,154.9,172.1,206.5,
    229.4,258.1,286.8]
num=0

for i in range(1,3):
    for j in range(0,12):
        nss_mcs_dict[(i,j)]=phy[num]
        num+=1

def get_phy(nss,mcs):
    return nss_mcs_dict[nss,mcs]

In [14]:
resource['phy_rate']=resource.apply(lambda row:get_phy(row['nss'],row['mcs']),axis=1)

In [15]:
resource=resource.drop(['nss','mcs'],axis=1)

In [16]:
resource.iloc[:,0]=resource.iloc[:,0].astype(str).map({'tcp':1,'udp':0})

In [17]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    650 non-null    object 
 1   bss_id                      650 non-null    int64  
 2   nav                         650 non-null    int64  
 3   ap_from_ap_0_sum_ant_rssi   650 non-null    object 
 4   ap_from_ap_1_sum_ant_rssi   650 non-null    object 
 5   sta_to_ap_0_sum_ant_rssi    650 non-null    object 
 6   sta_to_ap_1_sum_ant_rssi    650 non-null    object 
 7   sta_to_ap_2_sum_ant_rssi    650 non-null    object 
 8   sta_from_ap_0_sum_ant_rssi  650 non-null    object 
 9   sta_from_ap_1_sum_ant_rssi  650 non-null    object 
 10  sta_from_ap_2_sum_ant_rssi  650 non-null    object 
 11  sta_from_sta_0_rssi         650 non-null    float64
 12  sta_from_sta_1_rssi         650 non-null    float64
 13  per                         650 non

In [18]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [19]:
df=resource.iloc[:,range(18,24)]
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(df)

# 查看每个簇的平均特征值
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
# columns=['pro_pd', 'pro_ed', 'pro_nav']

print(f"Cluster centers:\n{cluster_centers}")

# 查看每个簇的样本数量
print(df['cluster'].value_counts())

D:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
D:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


Cluster centers:
          0         1         2         3         4         5
0  0.891473  0.002604  0.888165  0.905722  0.002657  0.925291
1  0.136728  0.002467  0.041090  0.157639  0.002483  0.092553
2  0.793557  0.001707  0.771345  0.300132  0.002526  0.119686
cluster
0    277
1    268
2    105
Name: count, dtype: int64


C:\Users\ShenFei\AppData\Local\Temp\ipykernel_20436\1928297672.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'] = kmeans.fit_predict(df)


In [20]:
resource['state']=df['cluster']

In [22]:
def dbm_to_linear(dbm):
    return 10 ** (dbm / 10)

def linear_to_dbm(linear):
    return 10 * np.log10(linear)

def add_dbm(dbm1, dbm2):
    # 使用 NumPy 的广播功能直接进行计算
    linear1 = dbm_to_linear(dbm1)
    linear2 = dbm_to_linear(dbm2)
    
    # 将两个线性功率值相加
    total_linear = linear1 + linear2
    
    # 将相加后的线性功率值转换回DBM
    total_dbm = linear_to_dbm(total_linear)
    
    return total_dbm


def func(x, y, z, state):
    if state == 0:
        return x + 100
    else:
        return x - add_dbm(y,z)

    
# 创建一个新列来存储 SINR 计算的结果
sinr = []
df = resource
# 遍历 DataFrame 的每一行
for i in range(len(df)):
    if df.iloc[i, list(df.columns).index('bss_id')] == 2:  # 偶数行
        result = func(df.iloc[i, list(df.columns).index('sta_from_ap_2_sum_ant_rssi')],df.iloc[i, list(df.columns).index('sta_from_ap_1_sum_ant_rssi')], df.iloc[i, list(df.columns).index('sta_from_ap_0_sum_ant_rssi')],df.iloc[i, list(df.columns).index('state')])
    elif df.iloc[i, list(df.columns).index('bss_id')] == 1:  # 奇数行
        result = func(df.iloc[i, list(df.columns).index('sta_from_ap_1_sum_ant_rssi')],df.iloc[i, list(df.columns).index('sta_from_ap_2_sum_ant_rssi')], df.iloc[i, list(df.columns).index('sta_from_ap_0_sum_ant_rssi')],df.iloc[i, list(df.columns).index('state')])
    else:
        result = func(df.iloc[i, list(df.columns).index('sta_from_ap_0_sum_ant_rssi')],df.iloc[i, list(df.columns).index('sta_from_ap_1_sum_ant_rssi')], df.iloc[i, list(df.columns).index('sta_from_ap_2_sum_ant_rssi')],df.iloc[i, list(df.columns).index('state')])
        
    sinr.append(result)

# 将计算结果添加为新列
df['sinr'] = sinr
resource = resource.drop('bss_id',axis=1)

In [32]:
resource = resource.loc[resource['sinr'] > 0]

In [33]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
Index: 627 entries, 0 to 649
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    627 non-null    object 
 1   nav                         627 non-null    int64  
 2   ap_from_ap_0_sum_ant_rssi   627 non-null    object 
 3   ap_from_ap_1_sum_ant_rssi   627 non-null    object 
 4   sta_to_ap_0_sum_ant_rssi    627 non-null    object 
 5   sta_to_ap_1_sum_ant_rssi    627 non-null    object 
 6   sta_to_ap_2_sum_ant_rssi    627 non-null    object 
 7   sta_from_ap_0_sum_ant_rssi  627 non-null    object 
 8   sta_from_ap_1_sum_ant_rssi  627 non-null    object 
 9   sta_from_ap_2_sum_ant_rssi  627 non-null    object 
 10  sta_from_sta_0_rssi         627 non-null    float64
 11  sta_from_sta_1_rssi         627 non-null    float64
 12  per                         627 non-null    float64
 13  num_ampdu                   627 non-null

In [24]:
# import pandas as pd
# from scipy import stats
# df = resource
# # 假设已有 DataFrame 'df'，包含 'state' 和 'sinr' 两列数据

# # 分别提取 state = 0 和 state ≠ 0 的 sinr 值
# sinr_state_0 = df[df['state'] == 0]['sinr']
# sinr_state_non_0 = df[df['state'] != 0]['sinr']

# # 进行独立样本t检验
# t_stat, p_value = stats.ttest_ind(sinr_state_0, sinr_state_non_0)

# # 输出结果
# print(f"T-statistic: {t_stat}, P-value: {p_value}")

# # 分析结果
# if p_value < 0.05:
#     print("sinr在state为0和state不为0的样本之间存在显著差异。")
#     if sinr_state_0.mean() > sinr_state_non_0.mean():
#         print("sinr在state为0时更大。")
#     else:
#         print("sinr在state为0时更小。")
# else:
#     print("sinr在state为0和state不为0的样本之间没有显著差异。")

T-statistic: 41.234043570168204, P-value: 2.4843000998793368e-183
sinr在state为0和state不为0的样本之间存在显著差异。
sinr在state为0时更大。


# read from here

In [7]:
#resource.to_excel('programe3_3ap_processed.xlsx')
import numpy as np
import pandas as pd
resource = pd.read_excel('programe3_3ap_processed.xlsx')
resource = resource.sample(frac=1).reset_index(drop=True)

# process

In [8]:
target=resource['throughput']
resource=resource.drop('throughput',axis=1)
resource=resource.drop('num_ampdu',axis=1)
resource=resource.drop('ppdu_dur',axis=1)

In [9]:
resource=resource.drop('sta_from_sta_0_rssi',axis=1)
resource=resource.drop('sta_from_sta_1_rssi',axis=1)

In [10]:
from sklearn.ensemble import RandomForestRegressor

rfg=RandomForestRegressor(n_estimators=100,criterion='friedman_mse',random_state=0,oob_score=False)


# target_0_phy=pd.DataFrame(target_0_phy)

rfg=rfg.fit(resource,target)

# rfg0=RandomForestRegressor(n_estimators=100,criterion='friedman_mse',random_state=0,oob_score=False)
# rfg0=rfg0.fit(resource,target_0.to_numpy())
from sklearn.model_selection import cross_val_score

res=cross_val_score(rfg,resource,target,cv=10)

In [11]:
res.mean()

0.96851192654992

In [12]:
res

array([0.98334349, 0.96686551, 0.95915152, 0.96409017, 0.95097542,
       0.98655284, 0.96545143, 0.98180721, 0.96531689, 0.96156478])

In [13]:
from xgboost import XGBRegressor as XGBR
reg=XGBR(n_estimators=100).fit(resource.to_numpy(),target.to_numpy())

In [14]:
res=cross_val_score(reg,resource.to_numpy(),target.to_numpy(),cv=10)

In [15]:
res

array([0.97810601, 0.958254  , 0.95584997, 0.97064261, 0.96462505,
       0.98477018, 0.95189116, 0.98281608, 0.97297215, 0.97991032])

In [16]:
res.mean()

0.9699837523609969

# predict  'programe3_test_3_3ap_predict.xlsx'

In [18]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    627 non-null    int64  
 1   nav                         627 non-null    int64  
 2   ap_from_ap_0_sum_ant_rssi   627 non-null    float64
 3   ap_from_ap_1_sum_ant_rssi   627 non-null    float64
 4   sta_to_ap_0_sum_ant_rssi    627 non-null    float64
 5   sta_to_ap_1_sum_ant_rssi    627 non-null    float64
 6   sta_to_ap_2_sum_ant_rssi    627 non-null    float64
 7   sta_from_ap_0_sum_ant_rssi  627 non-null    float64
 8   sta_from_ap_1_sum_ant_rssi  627 non-null    float64
 9   sta_from_ap_2_sum_ant_rssi  627 non-null    float64
 10  per                         627 non-null    float64
 11  seq_time                    627 non-null    float64
 12  pro_pd_0                    627 non-null    float64
 13  pro_ed_0                    627 non

In [19]:
import numpy as np
import pandas as pd
predict = pd.read_excel('programe3_test_3_3ap_predict.xlsx')


In [20]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    105 non-null    int64  
 1   nav                         105 non-null    int64  
 2   ap_from_ap_0_sum_ant_rssi   105 non-null    float64
 3   ap_from_ap_1_sum_ant_rssi   105 non-null    float64
 4   sta_to_ap_0_sum_ant_rssi    105 non-null    float64
 5   sta_to_ap_1_sum_ant_rssi    105 non-null    float64
 6   sta_to_ap_2_sum_ant_rssi    105 non-null    float64
 7   sta_from_ap_0_sum_ant_rssi  105 non-null    float64
 8   sta_from_ap_1_sum_ant_rssi  105 non-null    float64
 9   sta_from_ap_2_sum_ant_rssi  105 non-null    float64
 10  sta_from_sta_0_rssi         105 non-null    int64  
 11  sta_from_sta_1_rssi         105 non-null    int64  
 12  per                         105 non-null    float64
 13  num_ampdu                   0 non-n

In [21]:
predict=predict.drop('sta_from_sta_0_rssi',axis=1)
predict=predict.drop('sta_from_sta_1_rssi',axis=1)
predict=predict.drop('num_ampdu',axis=1)
predict=predict.drop('ppdu_dur',axis=1)

In [22]:
res=rfg.predict(predict)
pd.DataFrame(res).to_excel('programe3_test1_type3_res_throughput.xlsx')